In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import datetime
import calendar

from kiblib.utils.db import DbConn

In [2]:
db_conn = DbConn().create_engine()

In [3]:
year = 2025 #année pour les stats
month = 2 # Numéro du mois pour les stats

In [4]:
import locale
locale.setlocale(locale.LC_TIME,'')

'fr_FR.UTF-8'

In [5]:
month_name = calendar.month_name[month]
month_name

'février'

## TEST COMPTAGE JOURS OUVERTS 

In [6]:
query = f"SELECT * FROM statdb.stat_entrees WHERE MONTH(`datetime`) = {month} AND YEAR(`datetime`)={year}"

In [7]:
# On importe un dataframe qui correspond au mois et à l'année rentré plus haut ("month" et "year")
comptage_entrees = pd.read_sql(query,db_conn)
comptage_entrees

,datetime,entrees
0,2025-02-01 09:00:00,37
1,2025-02-01 10:00:00,105
2,2025-02-01 11:00:00,88
3,2025-02-01 12:00:00,85
4,2025-02-01 13:00:00,132
...,...,...
188,2025-02-28 15:00:00,119
189,2025-02-28 16:00:00,132
190,2025-02-28 17:00:00,89
191,2025-02-28 18:00:00,30


In [8]:
# On transforme le datetime en date, puis on aggrège les données par date
comptage_entrees['datetime'] = pd.DatetimeIndex(comptage_entrees['datetime']).date
comptage_entrees = comptage_entrees.groupby('datetime')['entrees'].sum().reset_index()

In [9]:
# Le nombre de jours ouverts = la somme du nombre de dates où on a une info
days = len(comptage_entrees) # nombre de jours ouverts (si pas de bug du compteur d'entrées)
days

20

In [10]:
comptage_entrees

,datetime,entrees
0,2025-02-01,1230
1,2025-02-02,532
2,2025-02-04,956
3,2025-02-05,1280
4,2025-02-06,894
5,2025-02-07,934
6,2025-02-08,1190
7,2025-02-11,1172
8,2025-02-12,1206
9,2025-02-13,984


# fin du test

In [11]:
# nb de personnes nouvellement inscrites
query = f"SELECT borrowernumber FROM koha_prod.borrowers WHERE YEAR(dateenrolled) = {year} AND MONTH(dateenrolled) = {month}"
nv_inscrits = pd.read_sql(query, db_conn)
nb_nv_inscrits = len(nv_inscrits)
nb_nv_inscrits

503

In [12]:
# nb de personnes  inscrites
query = f"SELECT COUNT(*) as inscrits FROM statdb.stat_adherents WHERE YEAR(date_extraction) = {year} AND MONTH(date_extraction) = {month}"
inscrits = pd.read_sql(query, db_conn)
nb_inscrits = inscrits['inscrits'][0]
print(nb_inscrits)


# entrées même mois un an plus tôt
query = f"SELECT COUNT(*) as inscrits FROM statdb.stat_adherents WHERE YEAR(date_extraction) = {year - 1} AND MONTH(date_extraction) = {month}"
inscrits = pd.read_sql(query, db_conn)
nb_inscrits_prec = inscrits['inscrits'][0]
print(nb_inscrits_prec)

# évolution
inscrits_evolution = round((nb_inscrits - nb_inscrits_prec) / nb_inscrits_prec * 100, 0)
print(inscrits_evolution)

15377
15114
2.0


In [13]:
# entrées
query = f"SELECT datetime, entrees FROM statdb.stat_entrees WHERE YEAR(datetime) = {year} AND MONTH(datetime) = {month}"
entrees = pd.read_sql(query, db_conn)
entrees['datetime'] = pd.to_datetime(entrees['datetime'])
nb_entrees = entrees['entrees'].sum()
print(nb_entrees)
max_entrees = entrees.groupby(entrees['datetime'].dt.date)['entrees'].sum().sort_values(ascending=False)
max_entrees_nb = max_entrees.iloc[0]
max_entrees_date = max_entrees.index[0]
print(max_entrees_nb)
print(max_entrees_date)

# entrées même mois un an plus tôt
query = f"SELECT datetime, entrees FROM statdb.stat_entrees WHERE YEAR(datetime) = {year - 1} AND MONTH(datetime) = {month}"
entrees = pd.read_sql(query, db_conn)
nb_entrees_prec = entrees['entrees'].sum()
print(nb_entrees_prec)

# évolution
entrees_evolution = round((nb_entrees - nb_entrees_prec) / nb_entrees_prec * 100, 0)
print(entrees_evolution)

21223
1499
2025-02-26
22060
-4.0


In [14]:
# prets
query = f"SELECT issuedate, itemnumber FROM statdb.stat_issues WHERE YEAR(issuedate) = {year} AND MONTH(issuedate) = {month} AND location != 'MED0A'"
prets = pd.read_sql(query, db_conn)
prets['issuedate'] = pd.to_datetime(prets['issuedate'])
nb_prets = len(prets)
print(nb_prets)
max_prets = prets.groupby(prets['issuedate'].dt.date).size().sort_values(ascending=False)
max_prets_nb = max_prets.iloc[0]
max_prets_date = max_prets.index[0]
print(max_prets_nb)
print(max_prets_date)

# même mois un an plus tôt
query = f"SELECT itemnumber FROM statdb.stat_issues WHERE YEAR(issuedate) = {year - 1} AND MONTH(issuedate) = {month} AND location != 'MED0A'"
prets = pd.read_sql(query, db_conn)
nb_prets_prec = len(prets)
print(nb_prets_prec)

# évolution
prets_evolution = round((nb_prets - nb_prets_prec) / nb_prets_prec * 100, 0)
print(prets_evolution)

27961
2446
2025-02-08
30513
-8.0


In [15]:
# connexions ordis
query = f"SELECT heure_deb FROM statdb.stat_webkiosk WHERE YEAR(heure_deb) = {year} AND MONTH(heure_deb) = {month}"
conn = pd.read_sql(query, db_conn)
conn['heure_deb'] = pd.to_datetime(conn['heure_deb'])
nb_conn = len(conn)
print(nb_conn)
max_conn = conn.groupby(conn['heure_deb'].dt.date).size().sort_values(ascending=False)
max_conn_nb = max_conn.iloc[0]
max_conn_date = max_conn.index[0]
print(max_conn_nb)
print(max_conn_date)

# même mois un an plus tôt
query = f"SELECT heure_deb FROM statdb.stat_webkiosk WHERE YEAR(heure_deb) = {year - 1} AND MONTH(heure_deb) = {month}"
conn = pd.read_sql(query, db_conn)
nb_conn_prec = len(conn)
print(nb_conn_prec)

# évolution
conn_evolution = round((nb_conn - nb_conn_prec) / nb_conn_prec * 100, 0)
print(conn_evolution)

4818
320
2025-02-26
5173
-7.0


In [16]:
# connexions wifi
query = f"SELECT start_wifi FROM statdb.stat_wifi WHERE YEAR(start_wifi) = {year} AND MONTH(start_wifi) = {month}"
wifi = pd.read_sql(query, db_conn)
nb_wifi = len(wifi)
print(nb_wifi)

# même mois un an plus tôt
query = f"SELECT start_wifi FROM statdb.stat_wifi WHERE YEAR(start_wifi) = {year - 1} AND MONTH(start_wifi) = {month}"
wifi = pd.read_sql(query, db_conn)
nb_wifi_prec = len(wifi)
print(nb_wifi_prec)

# évolution
wifi_evolution = round((nb_wifi - nb_wifi_prec) / nb_wifi_prec * 100, 0)
print(wifi_evolution)

# retour wifi le 14/10

1069
1152
-7.0


In [17]:
# réservations
query = f"SELECT reservedate FROM statdb.stat_reserves WHERE YEAR(reservedate) = {year} AND MONTH(reservedate) = {month}"
res = pd.read_sql(query, db_conn)
res['reservedate'] = pd.to_datetime(res['reservedate'])
nb_res = len(res)
print(nb_res)
max_res = res.groupby(res['reservedate'].dt.date).size().sort_values(ascending=False)
max_res_nb = max_res.iloc[0]
max_res_date = max_res.index[0]
print(max_res_nb)
print(max_res_date)

# même mois un an plus tôt
query = f"SELECT reservedate FROM statdb.stat_reserves WHERE YEAR(reservedate) = {year - 1} AND MONTH(reservedate) = {month}"
res = pd.read_sql(query, db_conn)
nb_res_prec = len(res)
print(nb_res_prec)

# évolution
res_evolution = round((nb_res - nb_res_prec) / nb_res_prec * 100, 0)
print(res_evolution)

4833
309
2025-02-01
4221
14.0


In [18]:
# site web
query = f"SELECT date, visites FROM statdb.stat_web2 WHERE YEAR(date) = {year} AND MONTH(date) = {month}"
web = pd.read_sql(query, db_conn)
nb_web = web['visites'].sum()
print(nb_web)


# même mois un an plus tôt
query = f"SELECT date, visites FROM statdb.stat_web2 WHERE YEAR(date) = {year - 1} AND MONTH(date) = {month}"
web = pd.read_sql(query, db_conn)
nb_web_prec = web['visites'].sum()
print(nb_web_prec)

# évolution
web_evolution = round((nb_web - nb_web_prec) / nb_web_prec * 100, 0)
print(web_evolution)

15648
15682
-0.0


In [19]:
# salle étude
query = f"SELECT datetime_entree FROM statdb.stat_freq_etude WHERE YEAR(datetime_entree) = {year} AND MONTH(datetime_entree) = {month}"
etude = pd.read_sql(query, db_conn)
nb_etude = len(etude)
print(nb_etude)

# même mois un an plus tôt
query = f"SELECT datetime_entree FROM statdb.stat_freq_etude WHERE YEAR(datetime_entree) = {year - 1} AND MONTH(datetime_entree) = {month}"
etude = pd.read_sql(query, db_conn)
nb_etude_prec = len(etude)
print(nb_etude_prec)

# évolution
etude_evolution = round((nb_etude - nb_etude_prec) / nb_etude_prec * 100, 0)
print(etude_evolution)

529
536
-1.0


In [24]:
print(f"Chiffres de la Mouette pour le mois de {month_name} {year}")

print("°"*65 +"\n")

print(f"En {month_name}, la médiathèque a été ouverte {days} jours au total.\n")

print(f"{nb_inscrits} personnes inscrites ({int(inscrits_evolution)} %).\n")

print("En moyenne, chaque jour (par rapport au même mois de l'année précédente) :\n")

print(f"{int(round(nb_entrees / days, 0))} entrées ({int(entrees_evolution)} %)\n")

#XXX prêts (-XX%)
print(f"{int(round(nb_prets / days, 0))} prêts ({int(prets_evolution)} %)\n")

#XXX connexions aux ordinateurs (+XX%)
print(f"{int(round(nb_conn / days, 0))} connexions aux ordinateurs ({int(conn_evolution)} %)\n")

#XXX connexions Wifi (-XX%)
print(f"{int(round(nb_wifi / days, 0))} connexions Wifi ({int(wifi_evolution)} %)\n")

#XXX réservations (-XX%)
print(f"{int(round(nb_res / days, 0))} réservations ({int(res_evolution)} %)\n")

#XXX visites du site internet (-XX%)
print(f"{int(round(nb_web / days, 0))} visites du site internet ({int(web_evolution)} %)\n")

#XX visites en salle d'études (-XX%)
print(f"{int(round(nb_etude / days, 0))} visites en salle d'études ({int(etude_evolution)} %)\n")

#XX visites du site web
print(f"{int(round(nb_web / days,0))} visites du site web ({int(web_evolution)}%)")

print("\n")
print("Les tops :")
print("°"*65 +"\n")

print(f"{max_entrees_nb} entrées le {max_entrees_date.strftime('%A %d %B %Y')}")
print(f"{max_conn_nb} connexions aux ordinateurs le {max_conn_date.strftime('%A %d %B %Y')}")
print (f"{max_res_nb} réservations le {max_res_date.strftime('%A %d %B %Y')}")
print(f"{max_prets_nb} prêts le {max_prets_date.strftime('%A %d %B %Y')}")

Chiffres de la Mouette pour le mois de février 2025
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

En février, la médiathèque a été ouverte 20 jours au total.

15377 personnes inscrites (2 %).

En moyenne, chaque jour (par rapport au même mois de l'année précédente) :

1061 entrées (-4 %)

1398 prêts (-8 %)

241 connexions aux ordinateurs (-7 %)

53 connexions Wifi (-7 %)

242 réservations (14 %)

782 visites du site internet (0 %)

26 visites en salle d'études (-1 %)

782 visites du site web (0%)


Les tops :
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

1499 entrées le mercredi 26 février 2025
320 connexions aux ordinateurs le mercredi 26 février 2025
309 réservations le samedi 01 février 2025
2446 prêts le samedi 08 février 2025
